In [23]:
import io
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import boto3
import sagemaker
from sagemaker import get_execution_role, PCA

In [24]:
session = sagemaker.Session()
print(session)

role = get_execution_role()
print(role)

bucket = session.default_bucket()

arn:aws:iam::240038582877:role/service-role/AmazonSageMaker-ExecutionRole-20191028T202433


### Customers

In [11]:
# load data
prefix = 'segmentation'

data_key= prefix + '/customers_scaled.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

arn:aws:iam::240038582877:role/service-role/AmazonSageMaker-ExecutionRole-20191028T202433


In [12]:
tmp_list = []

for chunk in pd.read_csv(data_location, chunksize=100000):
    tmp_list.append(chunk)
    
df = pd.concat(tmp_list, axis=0)
del tmp_list

In [19]:
# prepare space to store model artifacts
prefix = 'pca'
output_path = 's3://{}/{}/'.format(bucket, prefix)
output_path

's3://sagemaker-us-east-2-240038582877/pca/'

In [25]:
#current features less 1
N_COMPONENTS = 94
pca = PCA(role=role,
          train_instance_count=1,
          train_instance_type='ml.c4.xlarge',
          output_path=output_path,
          num_components=N_COMPONENTS,
          sagemaker_session=session)

In [26]:
#must convert to RecordSet for built in sagemaker models

#convert df to np array
train_data_np = df.values.astype('float32')

#convert to RecordSet format
formatted_train_data = pca.record_set(train_data_np)

In [27]:
pca.fit(formatted_train_data)

2020-04-07 03:45:18 Starting - Starting the training job...
2020-04-07 03:45:19 Starting - Launching requested ML instances...
2020-04-07 03:46:18 Starting - Preparing the instances for training......
2020-04-07 03:47:00 Downloading - Downloading input data...
2020-04-07 03:47:41 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/07/2020 03:47:57 INFO 140434131449664] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'_num_gpus': u'auto', u'_log_level': u'info', u'subtract_mean': u'true', u'force_dense': u'true', u'epochs': 1, u'algorithm_mode': u'regular', u'extra_components': u'-1', u'_kvstore': u'dist_sync', u'_num_kv_servers': u'auto'}
[04/07/2020 03:47:57 INFO 140434131449664] Reading provided configuration from /opt/ml/input/config/hyperparameters.json: {u'feature_dim': u'95', u'mini_batch_size': u'500', u'num_components':


2020-04-07 03:48:11 Uploading - Uploading generated training model
2020-04-07 03:48:11 Completed - Training job completed
Training seconds: 71
Billable seconds: 71


In [30]:
#get from AWS console
training_job_name = 'pca-2020-04-07-03-45-18-512'
model_key = os.path.join(prefix, training_job_name, 'output/model.tar.gz')
print(model_key)

boto3.resource('s3').Bucket(bucket).download_file(model_key, 'model.tar.gz')

os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

pca/pca-2020-04-07-03-45-18-512/output/model.tar.gz


2304

In [32]:
#!pip install mxnet
import mxnet as mx

pca_model_params = mx.ndarray.load('model_algo-1')

print(pca_model_params)

    100% |████████████████████████████████| 68.7MB 707kB/s eta 0:00:01
    100% |████████████████████████████████| 20.2MB 2.5MB/s eta 0:00:01
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
  Found existing installation: numpy 1.15.4
    Uninstalling numpy-1.15.4:
      Successfully uninstalled numpy-1.15.4
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
{'s': 
[1.02480202e+01 1.12598171e+01 1.24498568e+01 1.30026417e+01
 1.33955917e+01 1.51223946e+01 1.69041271e+01 1.79669075e+01
 1.94268093e+01 2.09470043e+01 2.22879772e+01 2.39021282e+01
 2.46324596e+01 2.78675594e+01 2.83099537e+01 2.93075581e+01
 3.24424362e+01 3.27273254e+01 3.71992226e+01 3.86713181e+01
 4.02246933e+01 4.08932953e+01 4.32484322e+01 4.51132050e+01
 4.81680641e+01 5.00800323e+01 5.15726128e+01 5.30764084e+01
 5.37760162e+01 5.47689095e+01 5.64503174e+01 5.81388779e+01
 5.88070641e+01 6.09319916e+01 6.267

In [ ]:
#mean - mean subtracted from a component in order to center it
#v - makeup of the principal component (same as components_)
# singular values of the components for the PCA transformation, % of variance from the projected future space